# Final project : Kiva Crownfunding


## TABLE OF CONTENTS

[Section 1](#section1)

   * [Q1.Normalize the loan_lenders table. Each row must have one loan_id and one lender](#section1.1)
   * [Q2.For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.](#section1.2)
   * [Q3.Find the lenders that have funded at least twice](#section1.3)
   * [Q4.For each country, compute how many loans have involved that country as borrowers](#section1.4)
   * [Q5.For each country, compute the overall amount of money borrowed](#section1.5)
   * [Q6.Like the previous point, but expressed as a percentage of the overall amount lent](#section1.6)
   * [Q7.Like the three previous points, but split for each year (with respect to disburse time)](#section1.7)
   * [Q8.For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount](#section1.8)
   * [Q9.For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.](#section1.9)
   * [Q10.Which country has the highest ratio between the difference computed at the previous point and the population?](#section1.10)
   * [Q11.Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?](#section1.11)
   * [Q12.For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018](#section1.12)
   

[Section 2](#section2)

   * [Q1.For each value of repayment_interval, add a new column to the lenders dataframe that contains the total amount of money corresponding to loans in such state](#section2.1)
   * [Q2.What is the occupation with the highest average amount of money lent (the average must be computed over all lenders with a given occupation)?](#section2.2)
   * [Q3.Cluster the loans according to the year-month of disburse time](#section2.3)
   * [Q4.For each country, compute its overall GNI, by multiplying the per capita GNI with its population](#section2.4)


[Section 3](#section3)

   * [Q1.Find the country with the highest rate of irregular repayment interval](#section3.1)
   * [Q2.Find the country with the highest fraction of loaned amount with irregular repayment interval](#section3.2)

<br>

### INTRODUCTION



--------------------------------------------------------------------
[Kiva.org](https://www.kiva.org/) is an online crowdfunding platform to extend financial services to poor and financially excluded people around the world. Kiva lenders have provided over $1 billion dollars in loans to over 2 million people. In order to set investment priorities, help inform lenders, and understand their target communities, knowing the level of poverty of each borrower is critical. However, this requires inference based on a limited set of information for each borrower.

For the locations in which Kiva has active loans, our objective is to pair Kiva's data with additional data sources to estimate the welfare level of borrowers in specific regions, based on shared economic and demographic characteristics. We are going to work on the [Kiva dataset](https://drive.google.com/file/d/1-tJtnIbo1Rt-F1XfoWGVkmBXiI-ciuRx/view). Some information on the datasets are available on the [Kaggle](https://www.kaggle.com/gaborfodor/undefined) web page.

--------------------------------------------------------------------

In [1]:
from platform import python_version

print(python_version())

3.7.5


### Read the Data

In [5]:
#current directory
import os
os.getcwd()

filePath = "./kiva"
#'change directory'
os.chdir(filePath)

#list elements of the dir
os.listdir()

['locations.csv',
 'loans_lenders.csv',
 'gmaps.png',
 'KadunaNigeria.png',
 'cover_small.png',
 'country_stats.csv',
 'progetto_da_caricare_su_github.py',
 'loan_coords.csv',
 'GEconV4.csv',
 'lenders.csv',
 'Leaderboard.png',
 'cover_v2.png',
 'loans.csv',
 '.ipynb_checkpoints']

In [3]:
#Load Libraries
import pandas as pd  
import numpy as np
from IPython.display import display
from datetime import datetime
import time
import warnings
warnings.filterwarnings('ignore')

In [21]:
#Additional kiva snapshot dataset:
import gc
gc.collect() 


loans_data = pd.read_csv('loans.csv')
lenders_data = pd.read_csv('lenders.csv')
loans_lenders_data = pd.read_csv('loans_lenders.csv')
country_stats_data = pd.read_csv('country_stats.csv')


<br>

<a id='section1'></a>

### Section 1

<a id='section1.1'></a>

##### Q1. Normalize the loan_lenders table. Each row must have one loan_id and one lender


In [20]:
#Let's look inside our data

#The loan lenders dataset shows 1387432 records and 2 columns 
#loan id, unique identifier is an integer  and lenders column a string 

display(loans_lenders_data.head())
display(loans_lenders_data.shape)
display(loans_lenders_data.info())


,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


(1387432, 2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1387432 entries, 0 to 1387431
Data columns (total 2 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   loan_id  1387432 non-null  int64 
 1   lenders  1387432 non-null  object
dtypes: int64(1), object(1)
memory usage: 21.2+ MB


None

<br>

First , we indexed the lenders column in order to remove the eventual blank spaces between lender names, and then we used the split method. The split method splits a string into a list where each word is a list item. For example for the following string

**txt = "lend1,lend2,lend3,lendr4" ->  x = txt.split(',')  ->  print(x) ->  ['lend1', 'lend2', 'lend3', 'lend4']**


Second, we use the [.explode()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.explode.html#pandas.DataFrame.explode) method that explodes lists into separate rows. 

In [9]:
import gc
gc.collect()    #clearing memory before next script execution

df1 = loans_lenders_data

df1.loc[:, 'lenders'] = df1.lenders.apply(lambda x: x.replace(' ', '').split(','))
loans_lenders_norm = df1.explode('lenders')
loans_lenders_norm.reset_index(drop=True, inplace=True)
display(loans_lenders_norm)

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


<br>

<a id='section1.2'></a>

##### Q2.For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [23]:
#Let's look inside the loans dataset

#The loans dataset shows 1419607 rows and 31 attributes 
#disburse time and  planned_expiration_time are strings, disburse time  with 1416794 observations and planned_expiration_time with 1047773 observations

display(loans_data.head())
display(loans_data.shape)


loans_data_Q2 = loans_data[["planned_expiration_time","disburse_time"]]
display(loans_data_Q2.info())

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner


(1419607, 31)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1419607 entries, 0 to 1419606
Data columns (total 2 columns):
 #   Column                   Non-Null Count    Dtype 
---  ------                   --------------    ----- 
 0   planned_expiration_time  1047773 non-null  object
 1   disburse_time            1416794 non-null  object
dtypes: object(2)
memory usage: 21.7+ MB


None

In [10]:
# checking missing data in loans data 
total = loans_data.isnull().sum().sort_values(ascending = False)
percent = (loans_data.isnull().sum()/loans_data.isnull().count()).sort_values(ascending = False)
missing_loans_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
display(missing_loans_data)


# check rows where disburse_time and planned_expiration_time are simultaneously NaN values
df2 = loans_data

display(df2[df2[['disburse_time','planned_expiration_time']].isna().all(axis=1)])

,Total,Percent
tags,784054,0.552304
description_translated,452678,0.318876
planned_expiration_time,371834,0.261927
currency_exchange_coverage_rate,321526,0.226489
town_name,161758,0.113946
raised_time,64282,0.045282
loan_name,47558,0.033501
description,43285,0.030491
loan_use,43273,0.030482
original_language,43251,0.030467


,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
423734,68814,NaN,NaN,NaN,NaN,0.0,475.0,refunded,Food Production/Sales,Food,...,NaN,12.0,0,1,1,NaN,NaN,NaN,monthly,field_partner
1129851,71582,NaN,NaN,NaN,NaN,50.0,1050.0,refunded,Retail,Retail,...,NaN,12.0,2,2,1,NaN,NaN,NaN,monthly,field_partner


As we can observe from the table above disburse_time and planned_expiration_time show 371834 and 2813 missing values, respectively(taking into consideration that they have in common 2 rows where they are simultaneously NaN). This means that the new column "Duration" if computed right, should have 374645 missing values. To have Duration as numbers, we divide its column of dtype timedelta by np.timedelta64(1, 'D') , but output is not int, but float, because when any of two dates is missing ( planned_expiration_time or disburse time) or both, also the duration must be [NaN values](https://pandas.pydata.org/pandas-docs/stable/user_guide/gotchas.html)

In [11]:
#convert string to datetime 

df2['planned_expiration_time']= pd.to_datetime(df2['planned_expiration_time']) 

df2['disburse_time']= pd.to_datetime(df2['disburse_time']) 

# create the Duration column as difference between planned exp time and disburse time

df2['Duration'] = df2['planned_expiration_time'].sub(df2['disburse_time'], axis=0)

# display the duration column 

df2['Duration'] = df2['Duration'] / np.timedelta64(1, 'D')

display(df2[['disburse_time','planned_expiration_time','Duration']])

# display how many NaN values in the duration column

display(df2['Duration'].isna().sum())


,disburse_time,planned_expiration_time,Duration
0,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,53.812569
1,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,96.600775
2,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,37.548669
3,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,34.798634
4,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,57.923634
...,...,...,...
1419602,2015-11-23 08:00:00+00:00,2016-01-02 01:00:03+00:00,39.708368
1419603,2015-11-24 08:00:00+00:00,2016-01-02 16:40:07+00:00,39.361192
1419604,2015-11-13 08:00:00+00:00,2016-01-03 22:20:04+00:00,51.597269
1419605,2015-11-03 08:00:00+00:00,2016-01-05 08:50:02+00:00,63.034745


374645

<br>

<a id='section1.3'></a>

##### Q3. Find the lenders that have funded at least twice.

In [27]:
#let's look inside the lenders data

display(lenders_data.head())

display(lenders_data.shape)

display(lenders_data.info())

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0


(2349174, 11)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2349174 entries, 0 to 2349173
Data columns (total 11 columns):
 #   Column             Dtype  
---  ------             -----  
 0   permanent_name     object 
 1   display_name       object 
 2   city               object 
 3   state              object 
 4   country_code       object 
 5   member_since       int64  
 6   occupation         object 
 7   loan_because       object 
 8   loan_purchase_num  float64
 9   invited_by         object 
 10  num_invited        int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 197.2+ MB


None

In [28]:
df3 = lenders_data.query('loan_purchase_num >= 2.0')['permanent_name']

#we observe that 877574 lenders have funded at least twice
display(df3)

1               reena6733
3          andrew86079135
6                rene7585
7              harald2826
11           jennifer4328
                ...      
2349158             rakhi
2349162     james75291930
2349164         carol8279
2349166      eric91401545
2349171         maria2141
Name: permanent_name, Length: 877574, dtype: object

<br>

<a id='section1.4'></a>

##### Q4. For each country, compute how many loans have involved that country as borrowers.

In [29]:
# we remember df2 = loans_data

df4 = df2.groupby(['country_code', 'country_name'])[['loan_id']].count().reset_index()

df4.rename(columns={'loan_id':'count_loan'}, inplace=True)

df4

,country_code,country_name,count_loan
0,AF,Afghanistan,2337
1,AL,Albania,3075
2,AM,Armenia,13952
3,AZ,Azerbaijan,10172
4,BA,Bosnia and Herzegovina,608
...,...,...,...
90,XK,Kosovo,2178
91,YE,Yemen,4206
92,ZA,South Africa,633
93,ZM,Zambia,1277


<br>

<a id='section1.5'></a>

##### Q5. For each country, compute the overall amount of money borrowed.

In [30]:
df5 = df2.groupby(['country_code','country_name'])['loan_amount'].sum().reset_index()

df5

,country_code,country_name,loan_amount
0,AF,Afghanistan,1967950.0
1,AL,Albania,4307350.0
2,AM,Armenia,22950475.0
3,AZ,Azerbaijan,14784625.0
4,BA,Bosnia and Herzegovina,477250.0
...,...,...,...
90,XK,Kosovo,3083025.0
91,YE,Yemen,3444000.0
92,ZA,South Africa,1006525.0
93,ZM,Zambia,1978975.0


<br>

<a id='section1.6'></a>

##### Q6. Like the previous point, but expressed as a percentage of the overall amount lent.

In [31]:
df5['percent'] = (df5['loan_amount']/df5['loan_amount'].sum())*100

df6 = df5[['country_code','country_name','loan_amount','percent']].copy(deep=True)

display(df6)


,country_code,country_name,loan_amount,percent
0,AF,Afghanistan,1967950.0,0.166578
1,AL,Albania,4307350.0,0.364598
2,AM,Armenia,22950475.0,1.942654
3,AZ,Azerbaijan,14784625.0,1.251452
4,BA,Bosnia and Herzegovina,477250.0,0.040397
...,...,...,...,...
90,XK,Kosovo,3083025.0,0.260964
91,YE,Yemen,3444000.0,0.291519
92,ZA,South Africa,1006525.0,0.085198
93,ZM,Zambia,1978975.0,0.167511


<br>

<a id='section1.7'></a>

##### Q7. Like the three previous points, but split for each year (with respect to disburse time).

In [32]:
df7 = df2[['country_code','country_name','loan_amount','loan_id','disburse_time']].copy(deep=True)

#create a new colum  "year_disburse_time" which extracts the year of the attribute "disburse_time"
df7['year_disburse_time'] =  df7['disburse_time'].dt.year.astype('Int64')


In [33]:
df7_1 = df7[['country_name','loan_id','year_disburse_time']].copy(deep=True)


# df7_1_1 shows  "how many loans that has involved countries as borrowers, splitting by year"

df7_1_1 = df7_1.groupby(['year_disburse_time'])[['loan_id']].count()
df7_1_1.rename(columns={'loan_id':'count_loan'}, inplace=True)

display(df7_1_1)


# df7_1_2 :  For each country , count loans which has involved that country as borrower, splitting by year.


df7_1_2 = df7_1.groupby([pd.Grouper(key='year_disburse_time'), 'country_name']).count().sort_index(ascending=True)
df7_1_2.rename(columns={'loan_id':'count_loan'}, inplace=True)

display(df7_1_2)

,count_loan
year_disburse_time,
2005,203
2006,2172
2007,24400
2008,54586
2009,83076
2010,93466
2011,114540
2012,133650
2013,140167


count_loan
year_disburse_time country_name            
2005               Bulgaria               3
                   Cambodia               2
                   Ecuador               10
                   Gaza                   8
                   Honduras              72
...                                     ...
2018               Tanzania              45
                   Thailand              13
                   Uganda                 2
                   Uruguay                1
                   Zambia                14

[748 rows x 1 columns]

In [47]:
#for a better visualization of df7_1_2 , we can reset the index
visual_countloan=  df7_1_2.reset_index()
visual_countloan

,year_disburse_time,country_name,count_loan
0,2005,Bulgaria,3
1,2005,Cambodia,2
2,2005,Ecuador,10
3,2005,Gaza,8
4,2005,Honduras,72
...,...,...,...
743,2018,Tanzania,45
744,2018,Thailand,13
745,2018,Uganda,2
746,2018,Uruguay,1


In [48]:
df7_2 = df7[['country_name','loan_amount','year_disburse_time']].copy(deep=True)

# df7_2_1 shows  "overall amount of money borrowed, splitting by year"

df7_2_1 = df7_2.groupby('year_disburse_time').sum()

display(df7_2_1)

# df7_2_2 : For each country, compute the overall amount of money borrowed , splitting by year

df7_2_2 = df7_2.groupby([pd.Grouper(key='year_disburse_time'), 'country_name']).sum().sort_index(ascending=True)

display(df7_2_2)

,loan_amount
year_disburse_time,
2005,102850.0
2006,1376575.0
2007,15446525.0
2008,39423050.0
2009,59689475.0
2010,72609150.0
2011,93699300.0
2012,119977575.0
2013,132043925.0


loan_amount
year_disburse_time country_name             
2005               Bulgaria           1850.0
                   Cambodia           1625.0
                   Ecuador            5000.0
                   Gaza               5000.0
                   Honduras          33300.0
...                                      ...
2018               Tanzania          14300.0
                   Thailand          26500.0
                   Uganda            21750.0
                   Uruguay            8000.0
                   Zambia            13975.0

[748 rows x 1 columns]

In [49]:

#Like the previous point, but expressed as a percentage of the overall amount lent, splitting by year

df7_3 = df7.groupby(['year_disburse_time'])['loan_amount'].sum().reset_index()

df7_3['percent'] = (df7_3['loan_amount']/df7_3['loan_amount'].sum())*100

df7_3


,year_disburse_time,loan_amount,percent
0,2005,102850.0,0.008787
1,2006,1376575.0,0.117606
2,2007,15446525.0,1.319656
3,2008,39423050.0,3.368062
4,2009,59689475.0,5.099500
5,2010,72609150.0,6.203278
6,2011,93699300.0,8.005090
7,2012,119977575.0,10.250143
8,2013,132043925.0,11.281018
9,2014,152270425.0,13.009045


<br>

<a id='section1.8'></a>

##### Q8. For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount..

In [50]:
df8 = df2[['loan_id','country_name','funded_amount','loan_amount','num_lenders_total']].copy(deep=True)

#assuming that all lenders contributed the same amount
df8['contrib_per_lender'] = df8['loan_amount'] / df8['num_lenders_total']


df8_2 = pd.merge(df8,loans_lenders_norm[['loan_id','lenders']],on='loan_id', how='left')

display(df8_2)


,loan_id,country_name,funded_amount,loan_amount,num_lenders_total,contrib_per_lender,lenders
0,657307,Philippines,125.0,125.0,3,41.666667,spencer5657
1,657307,Philippines,125.0,125.0,3,41.666667,matthew8640
2,657307,Philippines,125.0,125.0,3,41.666667,larry71496105
3,657259,Honduras,400.0,400.0,11,36.363636,ltr
4,657259,Honduras,400.0,400.0,11,36.363636,andrew5306
...,...,...,...,...,...,...,...
28326086,989240,Madagascar,175.0,175.0,7,25.000000,jensdamsgaardvanar
28326087,989240,Madagascar,175.0,175.0,7,25.000000,david47598776
28326088,989240,Madagascar,175.0,175.0,7,25.000000,bernd9221
28326089,989240,Madagascar,175.0,175.0,7,25.000000,valeria98599473


During the merge, NA groups in GroupBy are [automatically excluded](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html) we are matching the loan_id  in loans_data  which are simulnateous in loans_lenders_data . The loans_id which are not present in loans_lenders_data are automatically excluded because they don't have associated lenders (NaN). We are going to compute the w
hole contribution per each lender ( overall amount of money lent).


In [51]:
df8_3 = df8_2.groupby(['lenders'])['contrib_per_lender'].sum().reset_index()

display(df8_3)

,lenders,contrib_per_lender
0,000,1563.881085
1,00000,1249.947363
2,0002,2201.180463
3,00mike00,38.461538
4,0101craign0101,2424.088932
...,...,...
1383794,zzmcfate,60150.832762
1383795,zzpaghetti9994,44.247788
1383796,zzrvmf8538,551.829575
1383797,zzzsai,236.888048


In [52]:
# we can see for example, for the lenders spencer5657 ,  his total contribution ( we can perform it directly checking in the df8_3 dataframe)

display(df8_3.loc[(df8_3['lenders'] == 'spencer5657')])

#We have to also check lenders whose permanent name are 000, 00000,0002 ... because they look strange. We can do it by filtering 
#in the lenders_data table on permanent name . We can observe that the display name attribute indicates that they are true lenders

display(lenders_data.loc[(lenders_data['permanent_name'] == '000') | (lenders_data['permanent_name'] == '00000') | (lenders_data['permanent_name'] == '0002')])

,lenders,contrib_per_lender
1226670,spencer5657,84864.770248


,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
470721,0002,Lynn & Evan,Apple Valley,CA,US,1192667581,"Retired, Postmaster",To help those that want to help themselves.,68.0,NaN,0
569497,000,Chad Graves,Nashville,TN,US,1202872371,NaN,NaN,40.0,Vincent,0
887708,00000,Nanda,NaN,NaN,DE,1261131466,NaN,"Well, because I can ;o)\r\nThis world belongs ...",39.0,NaN,1


<br>

<a id='section1.9'></a>

##### Q9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

Firstly, we are going to merge  df8_2 (loans data + loans lenders norm )  and df3_1  (lenders data table )  on the lender key, taking into account the attributes 'funded_amount','loan_amount','contrib_per_lender' , and 'country code'  

In [53]:
#rename column permanent name in lenders data  as  'lenders'

df3_1 = lenders_data.rename(columns={'permanent_name':'lenders'})

#merge df8_2 and df3_1 as explained above

df9_1 = pd.merge(df8_2[['loan_id','funded_amount','loan_amount','contrib_per_lender','lenders']],df3_1[['lenders','country_code']],on='lenders', how='left')

display(df9_1)

,loan_id,funded_amount,loan_amount,contrib_per_lender,lenders,country_code
0,657307,125.0,125.0,41.666667,spencer5657,US
1,657307,125.0,125.0,41.666667,matthew8640,NaN
2,657307,125.0,125.0,41.666667,larry71496105,US
3,657259,400.0,400.0,36.363636,ltr,CA
4,657259,400.0,400.0,36.363636,andrew5306,US
...,...,...,...,...,...,...
28326086,989240,175.0,175.0,25.000000,jensdamsgaardvanar,NO
28326087,989240,175.0,175.0,25.000000,david47598776,US
28326088,989240,175.0,175.0,25.000000,bernd9221,DE
28326089,989240,175.0,175.0,25.000000,valeria98599473,LA


In [54]:
df9_1['country_code'].isna().sum()

5273556

As we can see above, the country code shows more than 5 million missing values. Since we assume that the distribution among the missing countries (NaN) is the same as the one computed from the rows where the country is known, we are going to use the **'ffill' DataFrameGroupBy.fillna method**  which propagates the last observed non-null value forward until another non-null value is encountered.


In [55]:
df9_2 = df9_1

col = ['country_code']

df9_2.loc[:,col] = df9_2.loc[:,col].ffill()


display(df9_2)

display(df9_2['country_code'].isna().sum())  #gives No missing values for country code


,loan_id,funded_amount,loan_amount,contrib_per_lender,lenders,country_code
0,657307,125.0,125.0,41.666667,spencer5657,US
1,657307,125.0,125.0,41.666667,matthew8640,US
2,657307,125.0,125.0,41.666667,larry71496105,US
3,657259,400.0,400.0,36.363636,ltr,CA
4,657259,400.0,400.0,36.363636,andrew5306,US
...,...,...,...,...,...,...
28326086,989240,175.0,175.0,25.000000,jensdamsgaardvanar,NO
28326087,989240,175.0,175.0,25.000000,david47598776,US
28326088,989240,175.0,175.0,25.000000,bernd9221,DE
28326089,989240,175.0,175.0,25.000000,valeria98599473,LA


0

Then , we can group by country and compute the overall amount for loan_amount and funded_amount


In [56]:
df9_3 = df9_2.groupby(['country_code']).agg({'funded_amount':'sum','loan_amount':'sum'}).reset_index()   

display(df9_3)

,country_code,funded_amount,loan_amount
0,AD,263125.0,264850.0
1,AE,100526935.0,102051875.0
2,AF,6977350.0,7116650.0
3,AG,32075.0,32075.0
4,AI,67950.0,67950.0
...,...,...,...
220,YE,913750.0,914550.0
221,YT,3275.0,3275.0
222,ZA,34704180.0,35033750.0
223,ZM,1999460.0,2063350.0


and finally, we can create the **diff amount column** which is the difference between the **overall amount of money lent** and the **overall amount of money borrowed** . 


In [57]:
df9_3['diff_amount'] = df9_3['loan_amount'].sub(df9_3['funded_amount'], axis=0)

display(df9_3)

,country_code,funded_amount,loan_amount,diff_amount
0,AD,263125.0,264850.0,1725.0
1,AE,100526935.0,102051875.0,1524940.0
2,AF,6977350.0,7116650.0,139300.0
3,AG,32075.0,32075.0,0.0
4,AI,67950.0,67950.0,0.0
...,...,...,...,...
220,YE,913750.0,914550.0,800.0
221,YT,3275.0,3275.0,0.0
222,ZA,34704180.0,35033750.0,329570.0
223,ZM,1999460.0,2063350.0,63890.0


<br>

<a id='section1.10'></a>

##### Q10. Which country has the highest ratio between the difference computed at the previous point and the population?



In [59]:
#Let's look inside the country_stats_data  table

#The loan lenders dataset shows 134 observations and 13 features

display(country_stats_data.head())
display(country_stats_data.shape)

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh


(174, 13)

In [60]:
#Let's merge df9_3 of the previous Question with country_stats_data, since we'll need the diff_amount computed.

df4 = country_stats_data

df10_1 = pd.merge(df9_3,df4[['country_code','country_name','population']],on='country_code', how='left')

display(df10_1)

,country_code,funded_amount,loan_amount,diff_amount,country_name,population
0,AD,263125.0,264850.0,1725.0,NaN,NaN
1,AE,100526935.0,102051875.0,1524940.0,United Arab Emirates,9400145.0
2,AF,6977350.0,7116650.0,139300.0,Afghanistan,35530081.0
3,AG,32075.0,32075.0,0.0,NaN,NaN
4,AI,67950.0,67950.0,0.0,NaN,NaN
...,...,...,...,...,...,...
220,YE,913750.0,914550.0,800.0,Yemen,28250420.0
221,YT,3275.0,3275.0,0.0,NaN,NaN
222,ZA,34704180.0,35033750.0,329570.0,South Africa,56717156.0
223,ZM,1999460.0,2063350.0,63890.0,Zambia,17094130.0


In [61]:
df10_2 = df10_1.dropna(subset=['country_name','population'])

#we calculate the ratio of diff_amount as ratio between diff_amount and the respective population

df10_2['ratio_diff_pop'] = df10_2['diff_amount'] / df10_2['population']


#reorder columns
df10_2 = df10_2[['country_code', 'country_name','funded_amount','loan_amount','diff_amount','population','ratio_diff_pop']]


display(df10_2.sort_values(by=['ratio_diff_pop'], ascending=False))

,country_code,country_name,funded_amount,loan_amount,diff_amount,population,ratio_diff_pop
149,NO,Norway,1.054297e+09,1.077115e+09,22818645.0,5305383.0,4.301036
13,AU,Australia,3.090068e+09,3.144695e+09,54627000.0,24450561.0,2.234182
207,US,United States,3.256134e+10,3.318281e+10,621467240.0,324459463.0,1.915393
35,CA,Canada,4.783952e+09,4.847605e+09,63653735.0,36624199.0,1.738024
148,NL,Netherlands,1.435755e+09,1.464832e+09,29077415.0,17035938.0,1.706828
...,...,...,...,...,...,...,...
219,XK,Kosovo,5.029250e+05,5.029250e+05,0.0,1895250.0,0.000000
23,BI,Burundi,1.812500e+04,1.812500e+04,0.0,10864245.0,0.000000
144,NE,Niger,5.600000e+03,5.600000e+03,0.0,21477348.0,0.000000
53,DJ,Djibouti,7.850000e+03,7.850000e+03,0.0,956985.0,0.000000


<br>

<a id='section1.11'></a>

##### Q11. Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?



In [62]:
#We calculate the population under the poverty line in the country stats data :

df4['population_under_poverty_line'] = df4['population'].sub((df4['population_below_poverty_line']/100)*df4['population'], axis=0)


display(df4)

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,population_under_poverty_line
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India,1.045900e+09
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria,5.726589e+07
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico,6.948984e+07
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan,1.388962e+08
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh,1.127988e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Somalia,SO,SOM,Africa,Eastern Africa,14742523,NaN,NaN,NaN,NaN,NaN,NaN,Somalia,NaN
170,Central African Republic,CF,CAF,Africa,Middle Africa,4659080,NaN,0.352440,51.458,7.098980,4.230000,587.473961,Central African Republic,NaN
171,Samoa,WS,WSM,Oceania,Polynesia,196440,NaN,0.702000,NaN,NaN,NaN,NaN,Samoa,NaN
172,Palestine,PS,PS,Asia,Western Asia,4920724,NaN,0.677000,NaN,NaN,NaN,NaN,Palestine,NaN


We notice that some countries do not fill the information containing the population_below_poverty_line . They are missing values , we are not going to ignore them for the purpose of the question 11 since on several web sources, including the [world health organization database](https://www.who.int/data/gho/data/themes/topics/indicator-groups/indicator-group-details/GHO/population-below-international-poverty-line) these specific countries don't have update information about the population below the poverty line.


In [63]:
#let's see the list of these specific countries

df4.loc[df4['population_below_poverty_line'].isnull()]

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,population_under_poverty_line
151,Australia,AU,AUS,Oceania,Australia and New Zealand,24450561,NaN,0.938680,82.537,20.43272,13.175100,42822.196270,Australia,NaN
152,Bahrain,BH,BHR,Asia,Western Asia,1492584,NaN,0.823884,76.715,14.50000,9.419000,37236.363040,Bahrain,NaN
153,Brunei,BN,BRN,Asia,South-eastern Asia,428697,NaN,0.864937,79.019,14.94486,9.020000,72843.466090,Brunei,NaN
154,Cuba,CU,CUB,Americas,Caribbean,11484636,NaN,0.774669,79.573,13.93739,11.750781,7455.062477,Cuba,NaN
155,Cyprus,CY,CYP,Asia,Western Asia,1179551,NaN,0.855646,80.332,14.25613,11.695230,29458.522690,Cyprus,NaN
156,Finland,FI,FIN,Europe,Northern Europe,5523231,NaN,0.894517,81.006,17.04792,11.194000,38868.135520,Finland,NaN
157,Iceland,IS,ISL,Europe,Northern Europe,335025,NaN,0.921110,82.724,18.99422,12.190892,37065.214510,Iceland,NaN
158,Kuwait,KW,KWT,Asia,Western Asia,4136528,NaN,0.800183,74.549,13.26000,7.265369,76075.207350,Kuwait,NaN
159,Luxembourg,LU,LUX,Europe,Western Europe,583455,NaN,0.898465,81.881,13.86373,11.950833,62470.591290,Luxembourg,NaN
160,New Zealand,NZ,NZL,Oceania,Australia and New Zealand,4705818,NaN,0.914900,82.026,19.22820,12.503000,32870.078360,New Zealand,NaN


In [64]:

#we merge result from question 9, df9_3  and df4 which contains the new attribute "population_under_poverty_line"

df11_1 = pd.merge(df9_3,df4[['country_code','country_name','population','population_under_poverty_line']],on='country_code', how='left')


#Now, we are going to show the country which has the highest ratio between the diff amount and the population under the poverty line

pd.options.mode.chained_assignment = None

df11_2 = df11_1.dropna(subset=['country_name','population_under_poverty_line'])

df11_2['ratio_diff_pop_underpov'] = df11_2['diff_amount'] / df11_2['population_under_poverty_line']

#reorder columns
df11_2 = df11_2[['country_code','country_name','funded_amount','loan_amount','diff_amount','population','population_under_poverty_line','ratio_diff_pop_underpov']]

#display values by descending order
display(df11_2.sort_values(by=['ratio_diff_pop_underpov'], ascending=False))

,country_code,country_name,funded_amount,loan_amount,diff_amount,population,population_under_poverty_line,ratio_diff_pop_underpov
207,US,United States,3.256134e+10,3.318281e+10,621467240.0,324459463.0,2.754661e+08,2.256057
35,CA,Canada,4.783952e+09,4.847605e+09,63653735.0,36624199.0,3.318152e+07,1.918349
148,NL,Netherlands,1.435755e+09,1.464832e+09,29077415.0,17035938.0,1.553678e+07,1.871522
54,DK,Denmark,2.895230e+08,2.969946e+08,7471625.0,5733551.0,4.965255e+06,1.504782
38,CH,Switzerland,6.112018e+08,6.228311e+08,11629305.0,8476005.0,7.916589e+06,1.468979
...,...,...,...,...,...,...,...,...
219,XK,Kosovo,5.029250e+05,5.029250e+05,0.0,1895250.0,1.326675e+06,0.000000
195,TL,Timor-Leste,7.595000e+04,7.595000e+04,0.0,1296311.0,7.544530e+05,0.000000
144,NE,Niger,5.600000e+03,5.600000e+03,0.0,21477348.0,1.172663e+07,0.000000
77,GM,Gambia,9.247500e+04,9.247500e+04,0.0,2100568.0,1.083893e+06,0.000000


United States of America (US) seems to be the country which the highest ratio between diff amount and the population under the poverty line.

<a id='section1.12'></a>

##### Q12. For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [69]:
#The calculation is based on the quantities evaluated at point 2

#select only loans with disburse_time  < planned_expiration_time
loans = df2.loc[df2.disburse_time < df2.planned_expiration_time, :]

min_year = (min(df2.disburse_time)).year
max_year = (max(df2.planned_expiration_time)).year
years_df = pd.DataFrame(index = range(min_year, max_year+1))

years_df['year_amount'] = 0


# loanAmount: amount of loans - from db
# timeColumn: considered date; it is planned_expiration_time or disburse_time, depending on which date is taken into account
# comparingDate: equals start or end of the considered year if incomplete periods are considered; equals 365 if complete periods are considered
# durationCol: column calculated in previous points (Q2), containing the duration (from disburse_time to planned_expiration_time)

def yearAmount(loanAmount, timeColumn, comparingDate, durationCol):
    
    if type(comparingDate) is int: #complete periods
        sumAmount = loanAmount * (comparingDate / durationCol)
    elif comparingDate.month == 12:  #incomplete period, starting period
        sumAmount = loanAmount * ((comparingDate - timeColumn).days)/ durationCol
    elif comparingDate.month == 1:  #incomplete period, ending period
        sumAmount = loanAmount * ((timeColumn - comparingDate).days)/ durationCol

    return sumAmount


for y in years_df.index:
    #starting period
    slicered_loans = loans.loc[(loans.disburse_time.dt.year == y) & (loans.Duration.notna()), :]
    if not slicered_loans.empty:
        tmpList = slicered_loans.apply(lambda x: yearAmount(x['loan_amount'], x['disburse_time'], pd.Timestamp(year = y, month = 12, day = 31, tz = 'UTC'), x['Duration']), axis = 1)
        years_df.loc[years_df.index == y, 'year_amount'] = years_df.loc[years_df.index == y, 'year_amount'] + sum(tmpList.values)

    #ending period
    slicered_loans = loans.loc[(loans.planned_expiration_time.dt.year == y) & (loans.Duration.notna()), :]
    if not slicered_loans.empty:
        tmpList = slicered_loans.apply(lambda x: yearAmount(x['loan_amount'], x['planned_expiration_time'], pd.Timestamp(year = y, month = 1, day = 1, tz = 'UTC'), x['Duration']), axis = 1)
        years_df.loc[years_df.index == y, 'year_amount'] = years_df.loc[years_df.index == y, 'year_amount'] + sum(tmpList.values)

    #middle periods
    slicered_loans = loans.loc[(((loans.planned_expiration_time.dt.year < y) & (loans.disburse_time.dt.year > y))) & (loans.Duration.notna()), :]
    if not slicered_loans.empty:
        dateYear = pd.Timestamp(year = y, month = 1, day = 1, tz = 'UTC')
        if dateYear.is_leap_year:
            numDays = 366
        else:
            numDays = 365
        tmpList = slicered_loans.apply(lambda x: yearAmount(x['loan_amount'], x['planned_expiration_time'], numDays, x['Duration']), axis = 1)
        years_df.loc[years_df.index == y, 'year_amount'] = years_df.loc[years_df.index == y, 'year_amount'] + sum(tmpList.values)



In [70]:
display(years_df)

,year_amount
2005,0.000000e+00
2006,0.000000e+00
2007,0.000000e+00
2008,0.000000e+00
2009,0.000000e+00
2010,0.000000e+00
2011,5.870466e+05
2012,9.675011e+08
2013,2.981345e+09
2014,1.323487e+10


<a id='section2'></a>

### Section 2

<a id='section2.1'></a>

##### Q1.For each value of repayment_interval, add a new column to the lenders dataframe that contains the total amount of money corresponding to loans in such state


In [24]:

#prendo i distinti valori di repayment_interval nel dataset loans
repayment = loans_data.repayment_interval.unique()

#si può sostituire --> repayment = loans_data.repayment_interval.unique()

#merge loans_lenders with loans - join = inner because we are interested only in matching rows (others have loan_amount 0 so they do not affect the sum)
merged_loans_lenders = pd.merge(loans_lenders_norm, loans_data, on='loan_id')
merged_loans_lenders.drop(merged_loans_lenders.columns.difference(['lenders', 'loan_id', 'loan_amount', 'repayment_interval']), 1, inplace = True)

#expand rows for each lender in 'lenders' column; then groupby loans_lenders by 'lenders' and 'repayment_interval', to have the sum of loan_amount per lender
#merged_loans_lenders = merged_loans_lenders.explode('lenders')
merged_loans_lenders = merged_loans_lenders.groupby(['lenders', 'repayment_interval']).agg({'loan_amount': 'sum'})

#Divide the result in columns and fill nan values with 0
merged_loans_lenders = merged_loans_lenders.unstack()
merged_loans_lenders = merged_loans_lenders.fillna(0)



# #merge with lenders table to add the requested columns (join = left outer, to maintain the number of records of the original table)
lenders_merged = pd.merge(lenders_data, merged_loans_lenders, left_on = 'permanent_name', right_index = True, how = 'left' )

display(lenders_merged)


,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited,"(loan_amount, bullet)","(loan_amount, irregular)","(loan_amount, monthly)","(loan_amount, weekly)"
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0,0.0,4075.0,0.0,0.0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0,0.0,5650.0,8800.0,0.0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0,0.0,0.0,10500.0,0.0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2349169,janet7309,Janet,NaN,NaN,NaN,1342097163,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
2349170,pj4198,NaN,NaN,NaN,NaN,1342097515,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
2349171,maria2141,Maria,NaN,NaN,US,1342099723,NaN,NaN,2.0,NaN,0,0.0,500.0,625.0,0.0
2349172,simone9846,Simone,NaN,NaN,NaN,1342100213,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN


<a id='section2.2'></a>

##### Q2.What is the occupation with the highest average amount of money lent (the average must be computed over all lenders with a given occupation)?

In [25]:
#Starting from the dataframe created at previous point (lenders_merged)

lenders_occupation = lenders_merged.loc[lenders_merged.occupation.notna(), :]


lenders_occupation['tot_amount'] = lenders_occupation.iloc[:, -1] + lenders_occupation.iloc[:, -2] + lenders_occupation.iloc[:, -3] + lenders_occupation.iloc[:, -4]
lenders_occupation_grouped = lenders_occupation.groupby('occupation').agg({'tot_amount': 'mean'})
lenders_occupation_grouped = lenders_occupation_grouped.sort_values(by = 'tot_amount', ascending = False)

#6 occupations with same tot_amount==highest tot_amount
display(lenders_occupation_grouped.head(6))

,tot_amount
occupation,
Faith | Care | Evangelism,233329375.0
www.linkedin.com/in/peacekeeper,230375375.0
"Retired Pharmacist, Business Owner and Investor",121142700.0
Making the coolest belts around. No Holes. Always Fits.,87185925.0
Cat herder,57670800.0
Vice President Client Service,56871550.0


<a id='section2.3'></a>

##### Q3.Cluster the loans according to the year-month of disburse time

In [27]:
#NB: starting from loans.csv file

loans_data.loc[loans_data.disburse_time.isna(), 'disburse_time'] = pd.NaT
loans_data.loc[loans_data.disburse_time.notna(), 'disburse_time'] = pd.to_datetime(loans_data.disburse_time)

loans_cluster = loans_data.groupby([loans_data.disburse_time.dt.year, loans_data.disburse_time.dt.month])['loan_id'].apply(list)
display(loans_cluster)


disburse_time  disburse_time
2005.0         4.0              [89, 241, 143, 231, 173, 171, 133, 108, 155, 2...
2006.0         3.0                                                          [147]
               5.0              [406, 350, 399, 377, 349, 313, 347, 317, 312, ...
               6.0              [457, 439, 436, 470, 438, 372, 477, 393, 448, ...
               7.0              [480, 514, 532, 497, 527, 390, 494, 548, 559, ...
                                                      ...                        
2017.0         11.0             [1427320, 1429865, 1432025, 1433781, 1435004, ...
               12.0             [1433150, 1435040, 1441153, 1441165, 1436592, ...
2018.0         1.0              [1444070, 1432653, 1432432, 1432647, 1402001, ...
               2.0              [1418468, 1411544, 1431631, 1431467, 1435193, ...
               3.0              [1429907, 1429527, 1429565, 1429195, 1429510, ...
Name: loan_id, Length: 145, dtype: object


<a id='section2.4'></a>

##### Q4.For each country, compute its overall GNI, by multiplying the per capita GNI with its population

In [28]:
#country_stats_data = pd.read_csv(filePath + 'country_stats.csv', header = 0)

#explore df
display(country_stats_data.info())

#check if there are duplicates
display('Df records: ', len(country_stats_data), '\nUnique Country names: ', len(country_stats_data.country_name.unique()))

#select only useful columns
country = country_stats_data.loc[:, ['country_name', 'population', 'gni']]

#check nan values
display('Country NaNs: ', country.country_name.isna().sum(), '\nPopulation NaNs: ', country.population.isna().sum(), '\nGNI NaNs: ', country.gni.isna().sum())

#6 values are NaNs in GNI column - show these records:
display(country.loc[country.gni.isna(), :])

#replace nan values to avoid nan overall gni values
country.loc[:, 'gni'] = country.gni.fillna(0)
country['overall_gni'] = country.population * country.gni

display(country)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   country_name                   174 non-null    object 
 1   country_code                   173 non-null    object 
 2   country_code3                  173 non-null    object 
 3   continent                      174 non-null    object 
 4   region                         174 non-null    object 
 5   population                     174 non-null    int64  
 6   population_below_poverty_line  152 non-null    float64
 7   hdi                            171 non-null    float64
 8   life_expectancy                168 non-null    float64
 9   expected_years_of_schooling    168 non-null    float64
 10  mean_years_of_schooling        168 non-null    float64
 11  gni                            168 non-null    float64
 12  kiva_country_name              174 non-null    obj

None

'Df records: '

174

'\nUnique Country names: '

...

0

'\nGNI NaNs: '

6

,country_name,population,gni
161,North Korea,25490965,NaN
164,Puerto Rico,3663131,NaN
169,Somalia,14742523,NaN
171,Samoa,196440,NaN
172,Palestine,4920724,NaN
173,Kosovo,1895250,NaN


,country_name,population,gni,overall_gni
0,India,1339180127,5663.474799,7.584413e+12
1,Nigeria,190886311,5442.901264,1.038975e+12
2,Mexico,129163276,16383.106680,2.116096e+12
3,Pakistan,197015955,5031.173074,9.912214e+11
4,Bangladesh,164669751,3341.490722,5.502424e+11
...,...,...,...,...
169,Somalia,14742523,0.000000,0.000000e+00
170,Central African Republic,4659080,587.473961,2.737088e+09
171,Samoa,196440,0.000000,0.000000e+00
172,Palestine,4920724,0.000000,0.000000e+00


<br>

<a id='section3'></a>

### Section 3

<a id='section3.1'></a>

##### Q1.Find the country with the highest rate of irregular repayment interval.

In [31]:
#NB: starting from loans_data dataframe

display('NaN values for repayment_interval: ', loans_data.repayment_interval.isna().sum())
## First compute the total number of loans per country (no nan values for repayment_interval)
tot_loans = loans_data.loc[:, ['loan_id', 'country_name']]
loans_grouped_by_country = tot_loans.groupby('country_name').agg({'loan_id': 'count'})

#Then select only loans with irregular repayment interval
irregular_repayment_loans = loans_data.loc[loans_data.repayment_interval == 'irregular', ['loan_id', 'country_name']]

#group by country, counting the loan_id values for each country, to obtain the count of irregular repayment intervals per country
irregular_grouped_by_country = irregular_repayment_loans.groupby('country_name').agg({'loan_id': 'count'})
country_irregular_rate = pd.merge(irregular_grouped_by_country, loans_grouped_by_country, on='country_name', how='left')
country_irregular_rate.columns = ['irregular_loans_count', 'loans_count']
country_irregular_rate['irregular_rate'] = country_irregular_rate.irregular_loans_count / country_irregular_rate.loans_count

#countries with max rate of irregular payment intervals:
country_irregular_rate = country_irregular_rate.sort_values(by = 'irregular_rate', ascending = False)

display(country_irregular_rate)

display(country_irregular_rate.loc[country_irregular_rate.irregular_rate == 1, :].index)

'NaN values for repayment_interval: '

0

,irregular_loans_count,loans_count,irregular_rate
country_name,,,
Solomon Islands,903,903,1.000000
Papua New Guinea,1,1,1.000000
Canada,1,1,1.000000
Vanuatu,4,4,1.000000
Uruguay,1,1,1.000000
...,...,...,...
Jordan,10,10161,0.000984
Afghanistan,1,2337,0.000428
Egypt,1,2460,0.000407


Index(['Solomon Islands', 'Papua New Guinea', 'Canada', 'Vanuatu', 'Uruguay',
       'Bhutan', 'Sri Lanka', 'Mauritania', 'Botswana'],
      dtype='object', name='country_name')

<a id='section3.2'></a>

##### Q2.Find the country with the highest fraction of loaned amount with irregular repayment interval

In [32]:
#NB: starting from loans_data


display('NaN values for repayment_interval: ', loans_data.repayment_interval.isna().sum())
## First compute the total loaned amount per country (no nan values for repayment_interval)
tot_loans = loans_data.loc[:, ['loan_amount', 'country_name']]
loans_grouped_by_country = tot_loans.groupby('country_name').agg({'loan_amount': 'sum'})

#Then select only loans with irregular repayment interval
irregular_repayment_loans = loans_data.loc[loans_data.repayment_interval == 'irregular', ['loan_amount', 'country_name']]

#group by country, summing the loan_amount values for each country
irregular_grouped_by_country = irregular_repayment_loans.groupby('country_name').agg({'loan_amount': 'sum'})
country_irregular_amount = pd.merge(irregular_grouped_by_country, loans_grouped_by_country, on='country_name', how='left')
country_irregular_amount.columns = ['irregular_loans_amount', 'loans_amount']
country_irregular_amount['irregular_amount_fraction'] = country_irregular_amount.irregular_loans_amount / country_irregular_amount.loans_amount

#countries with max rate of irregular payment intervals:
country_irregular_amount = country_irregular_amount.sort_values(by = 'irregular_amount_fraction', ascending = False)

display(country_irregular_amount)

display(country_irregular_amount.loc[country_irregular_amount.irregular_amount_fraction == max(country_irregular_amount.irregular_amount_fraction), :].index)

'NaN values for repayment_interval: '

0

,irregular_loans_amount,loans_amount,irregular_amount_fraction
country_name,,,
Mauritania,15000.0,15000.0,1.000000
Botswana,8000.0,8000.0,1.000000
Sri Lanka,74800.0,74800.0,1.000000
Canada,50000.0,50000.0,1.000000
Papua New Guinea,50000.0,50000.0,1.000000
...,...,...,...
Egypt,1925.0,1415025.0,0.001360
Palestine,21825.0,22316675.0,0.000978
Jordan,7850.0,11007450.0,0.000713


Index(['Mauritania', 'Botswana', 'Sri Lanka', 'Canada', 'Papua New Guinea',
       'Solomon Islands', 'Bhutan', 'Uruguay', 'Vanuatu'],
      dtype='object', name='country_name')